## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [3]:
# Get today's date
today = datetime.now() - timedelta(days=2)

# Get the date one week ago
one_week_ago = today - timedelta(days=8)

def format_date(date):
    day = date.day
    # Months 3-7 (March–July) should be full; all others abbreviated with a period
    if date.month in (3, 4, 5, 6, 7):
        month_str = date.strftime("%B")      # e.g. "March", "April", ...
    else:
        month_str = date.strftime("%b") + "."  # e.g. "Jan.", "Feb.", "Aug.", ...

    return f"{month_str} {day}"


# Format both dates
week_ending = format_date(today)
week_starting = format_date(one_week_ago)

print(week_starting, week_ending)

May 30 June 7


In [4]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [5]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [6]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [7]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [8]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,June-5-2025,Condo/Co-op,4475 N Ocean Blvd Unit 21d,Delray Beach,FL,33483.0,1532500.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/4475-N-...,Beaches MLS,RX-11088899,N,Y,26.503852,-80.053038


In [9]:
print('Input county name:')
county = input()

Input county name:


 Palm Beach


In [10]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [11]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [12]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [13]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-06-02    30
2025-06-03    17
2025-06-05    16
2025-06-06    14
2025-06-04    14
Name: count, dtype: int64

In [14]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [15]:
df_top_ten = df2.head(10)

In [16]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
30,PAST SALE,2025-06-03,Condo/Co-op,525 S Flagler Dr Ph 3ab,West Palm Beach,FL,33401.0,6050000.0,3.0,4.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/525-...,Beaches MLS,RX-11079097,N,Y,26.708264,-80.051289
12,PAST SALE,2025-06-04,Condo/Co-op,20155 Boca West Dr Unit C601,Boca Raton,FL,33434.0,5300000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20155-Boc...,Beaches MLS,RX-11076823,N,Y,26.377468,-80.163463
66,PAST SALE,2025-06-04,Condo/Co-op,525 S Flagler Dr Unit 21 C/D,West Palm Beach,FL,33401.0,4900000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/525-...,Beaches MLS,RX-11040645,N,Y,26.708413,-80.051135
65,PAST SALE,2025-06-02,Condo/Co-op,2700 N Ocean Dr Unit 2502b,Riviera Beach,FL,33404.0,3100000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-11053568,N,Y,26.785118,-80.033251
71,PAST SALE,2025-06-03,Condo/Co-op,2727 Ocean Blvd #407,Highland Beach,FL,33487.0,2750000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/2727-...,Beaches MLS,F10477372,N,Y,26.424944,-80.063184
1,PAST SALE,2025-06-05,Condo/Co-op,4475 N Ocean Blvd Unit 21d,Delray Beach,FL,33483.0,1532500.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/4475-N-...,Beaches MLS,RX-11088899,N,Y,26.503852,-80.053038
19,PAST SALE,2025-06-04,Condo/Co-op,86 Macfarlane Dr Unit 5b,Delray Beach,FL,33483.0,1520000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/86-Macf...,Beaches MLS,RX-11063198,N,Y,26.460259,-80.063221
77,PAST SALE,2025-06-03,Condo/Co-op,3000 S Ocean Blvd #1501,Boca Raton,FL,33432.0,1250000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/3000-S-Oc...,Beaches MLS,F10474288,N,Y,26.321745,-80.075232
10,PAST SALE,2025-06-03,Condo/Co-op,5150 N Ocean Dr #1803,Singer Island,FL,33404.0,1180000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5150-N...,Beaches MLS,RX-11081451,N,Y,26.803026,-80.033425
31,PAST SALE,2025-06-03,Condo/Co-op,2003 N Ocean Blvd #302,Boca Raton,FL,33431.0,985000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2003-N-Oc...,Beaches MLS,RX-11069722,N,Y,26.369141,-80.070100


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [17]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [18]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [19]:
response_df = pd.DataFrame(response_list)

In [20]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [21]:
df_top_ten = merged_df

In [22]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-06-03,Condo/Co-op,525 S Flagler Dr Ph 3ab,West Palm Beach,FL,33401.0,6050000.0,3.0,4.0,...,-80.051289,https://www.redfin.com/FL/West-Palm-Beach/525-...,Samantha Curry,Douglas Elliman,None,None,Samantha Curry,Douglas Elliman,None,None
1,PAST SALE,2025-06-04,Condo/Co-op,20155 Boca West Dr Unit C601,Boca Raton,FL,33434.0,5300000.0,3.0,3.5,...,-80.163463,https://www.redfin.com/FL/Boca-Raton/20155-Boc...,Rebecca Spooner,Siemens Group Realty Corp.,James Spooner,Siemens Group Realty Corp.,Judith Romanow,Boca West Realty LLC,None,None
2,PAST SALE,2025-06-04,Condo/Co-op,525 S Flagler Dr Unit 21 C/D,West Palm Beach,FL,33401.0,4900000.0,3.0,4.5,...,-80.051135,https://www.redfin.com/FL/West-Palm-Beach/525-...,Christopher Leavitt,Douglas Elliman,None,None,Valentina Kraver,Douglas Elliman,None,None
3,PAST SALE,2025-06-02,Condo/Co-op,2700 N Ocean Dr Unit 2502b,Riviera Beach,FL,33404.0,3100000.0,3.0,3.5,...,-80.033251,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Patrick Letourneau,One Sotheby's International Re,None,None,Michelle Rinaldi,EXP Realty LLC,None,None
4,PAST SALE,2025-06-03,Condo/Co-op,2727 Ocean Blvd #407,Highland Beach,FL,33487.0,2750000.0,3.0,2.5,...,-80.063184,https://www.redfin.com/FL/Highland-Beach/2727-...,Stephen Clementi,Coldwell Banker Realty,None,None,None,Sun Realty,None,None
5,PAST SALE,2025-06-05,Condo/Co-op,4475 N Ocean Blvd Unit 21d,Delray Beach,FL,33483.0,1532500.0,2.0,2.0,...,-80.053038,https://www.redfin.com/FL/Delray-Beach/4475-N-...,Bettina Smith,The Corcoran Group,None,None,Bettina Smith,The Corcoran Group,None,None
6,PAST SALE,2025-06-04,Condo/Co-op,86 Macfarlane Dr Unit 5b,Delray Beach,FL,33483.0,1520000.0,3.0,3.0,...,-80.063221,https://www.redfin.com/FL/Delray-Beach/86-Macf...,Bettina Smith,The Corcoran Group,None,None,Bettina Smith,The Corcoran Group,None,None
7,PAST SALE,2025-06-03,Condo/Co-op,3000 S Ocean Blvd #1501,Boca Raton,FL,33432.0,1250000.0,3.0,2.0,...,-80.075232,https://www.redfin.com/FL/Boca-Raton/3000-S-Oc...,Joseph Hillner,Guaranteed Home Sales,None,None,Joyce Marie Schneider,"Castles By The Beach, Inc.",None,None
8,PAST SALE,2025-06-03,Condo/Co-op,5150 N Ocean Dr #1803,Singer Island,FL,33404.0,1180000.0,2.0,2.0,...,-80.033425,https://www.redfin.com/FL/Riviera-Beach/5150-N...,Angela Voland,Waterfront Properties & Club C,None,None,Jonathan Hazman,Echo Fine Properties,None,None
9,PAST SALE,2025-06-03,Condo/Co-op,2003 N Ocean Blvd #302,Boca Raton,FL,33431.0,985000.0,3.0,2.5,...,-80.070100,https://www.redfin.com/FL/Boca-Raton/2003-N-Oc...,Lisa Nanfra,Compass Florida LLC,None,None,Deena Padnis,Re/Max 1st Choice,None,None


## Current Week's Values

In [23]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

91


In [24]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$592,601


In [25]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$341


In [26]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$53,926,699


In [27]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0           91              592601          341                     53926699.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [28]:
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input Previous Week Condo Sales Total (number units sold):
129
-----------
Input Previous Week Condo Average Sales Price:
485614
-----------
Input Previous Week Condo Average PSF:
323
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
62644186.0


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [29]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [30]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [31]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [32]:
pd.set_option('display.max_columns',None)

In [33]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-06-03,Condo/Co-op,525 S Flagler Dr Ph 3ab,West Palm Beach,FL,33401.0,6050000.0,3.0,4.0,Plaza OF The Palm Beaches Condo,4018.0,NaN,1986.0,NaN,1506.0,5986.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/525-...,Beaches MLS,RX-11079097,N,Y,26.708264,-80.051289,https://www.redfin.com/FL/West-Palm-Beach/525-...,Samantha Curry,Douglas Elliman,None,None,Samantha Curry,Douglas Elliman,None,None,1,orange


In [34]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [35]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-06-03,Condo/Co-op,525 S Flagler Dr Ph 3ab,West Palm Beach,FL,33401.0,6050000.0,3.0,4.0,Plaza OF The Palm Beaches Condo,4018.0,NaN,1986.0,NaN,1506.0,5986.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/525-...,Beaches MLS,RX-11079097,N,Y,26.708264,-80.051289,https://www.redfin.com/FL/West-Palm-Beach/525-...,Samantha Curry,Douglas Elliman,None,None,Samantha Curry,Douglas Elliman,None,None,1,orange
1,PAST SALE,2025-06-04,Condo/Co-op,20155 Boca West Dr Unit C601,Boca Raton,FL,33434.0,5300000.0,3.0,3.5,Akoya Boca West,2873.0,NaN,2019.0,NaN,1845.0,5133.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20155-Boc...,Beaches MLS,RX-11076823,N,Y,26.377468,-80.163463,https://www.redfin.com/FL/Boca-Raton/20155-Boc...,Rebecca Spooner,Siemens Group Realty Corp.,James Spooner,Siemens Group Realty Corp.,Judith Romanow,Boca West Realty LLC,None,None,2,blue
2,PAST SALE,2025-06-04,Condo/Co-op,525 S Flagler Dr Unit 21 C/D,West Palm Beach,FL,33401.0,4900000.0,3.0,4.5,Plaza OF The Palm Beaches Condo,4018.0,NaN,1986.0,NaN,1220.0,5853.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/525-...,Beaches MLS,RX-11040645,N,Y,26.708413,-80.051135,https://www.redfin.com/FL/West-Palm-Beach/525-...,Christopher Leavitt,Douglas Elliman,None,None,Valentina Kraver,Douglas Elliman,None,None,3,blue
3,PAST SALE,2025-06-02,Condo/Co-op,2700 N Ocean Dr Unit 2502b,Riviera Beach,FL,33404.0,3100000.0,3.0,3.5,2700 North Ocean Condo,2660.0,NaN,2008.0,NaN,1165.0,3942.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-11053568,N,Y,26.785118,-80.033251,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Patrick Letourneau,One Sotheby's International Re,None,None,Michelle Rinaldi,EXP Realty LLC,None,None,4,blue
4,PAST SALE,2025-06-03,Condo/Co-op,2727 Ocean Blvd #407,Highland Beach,FL,33487.0,2750000.0,3.0,2.5,Villa Magna Condo,2510.0,NaN,1982.0,NaN,1096.0,2823.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/2727-...,Beaches MLS,F10477372,N,Y,26.424944,-80.063184,https://www.redfin.com/FL/Highland-Beach/2727-...,Stephen Clementi,Coldwell Banker Realty,None,None,None,Sun Realty,None,None,5,blue
5,PAST SALE,2025-06-05,Condo/Co-op,4475 N Ocean Blvd Unit 21d,Delray Beach,FL,33483.0,1532500.0,2.0,2.0,ST Andrews Ocean Condo,1520.0,NaN,1973.0,NaN,1008.0,2357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/4475-N-...,Beaches MLS,RX-11088899,N,Y,26.503852,-80.053038,https://www.redfin.com/FL/Delray-Beach/4475-N-...,Bettina Smith,The Corcoran Group,None,None,Bettina Smith,The Corcoran Group,None,None,6,blue
6,PAST SALE,2025-06-04,Condo/Co-op,86 Macfarlane Dr Unit 5b,Delray Beach,FL,33483.0,1520000.0,3.0,3.0,Bar Harbour Condo,2363.0,NaN,1967.0,NaN,643.0,2887.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/86-Macf...,Beaches MLS,RX-11063198,N,Y,26.460259,-80.063221,https://www.redfin.com/FL/Delray-Beach/86-Macf...,Bettina Smith,The Corcoran Group,None,None,Bettina Smith,The Corcoran Group,None,None,7,blue
7,PAST SALE,2025-06-03,Condo/Co-op,3000 S Ocean Blvd #1501,Boca Raton,FL,33432.0,1250000.0,3.0,2.0,3000 South,1782.0,NaN,1970.0,NaN,701.0,2689.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/3000-S-Oc...,Beaches MLS,F10474288,N,Y,26.321745,-80.075232,https://www.redfin.com/FL/Boca-Raton/3000-S-Oc...,Joseph Hillner,Guaranteed Home Sales,None,None,Joyce Marie Schneider,"Castles By The Beach, Inc.",None,None,8,blue
8,PAST SALE,2025-06-03,Condo/Co-op,5150 N Ocean Dr #1803,Singer Island,FL,33404.0,1180000.0,2.0,2.0,Via Delfino,2088.0,NaN,2003.0,NaN,565.0,2150.0,Sol

In [36]:
%store -r map_box_api_key

In [38]:
import folium
from branca.element import Template, MacroElement

# ——— Your title & caption HTML ———
title_html = f'''
  <h3 align="center" style="font-size:16px"><b>Recent {county} County Condo Sales</b></h3>
'''
caption_html = f'''
  <p align="center" style="font-size:13px"><i>{week_starting} — {week_ending}</i></p>
'''

# ——— Base map ———
m = folium.Map(
    location=df[['LATITUDE','LONGITUDE']].mean().to_list(),
    zoom_start=9,
    tiles=None
)

# ——— Tile layers ———
folium.TileLayer('OpenStreetMap', name='OSM', control=False).add_to(m)
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox", name="Streets", control=False
).add_to(m)

# ——— Add your markers ———
for _, row in df_top_ten.iterrows():
    lat, lon = row['LATITUDE'], row['LONGITUDE']
    popup = folium.Popup(popup_html(row), max_width=400)

    if row['COLOR'] == 'blue':
        # Regular sale: blue circle
        folium.CircleMarker(
            location=(lat, lon),
            radius=7,
            fill=True,
            fill_color='blue',
            fill_opacity=0.5,
            color=None,
            popup=popup
        ).add_to(m)
    else:
        # Top sale: gold star
        folium.Marker(
            location=(lat, lon),
            popup=popup,
            icon=folium.DivIcon(
                html="""
                <div style="font-size:24px; line-height:24px; color:gold;">
                  &#9733;
                </div>
                """
            )
        ).add_to(m)

# ——— Layer control, title & caption ———
folium.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

# ——— Optional legend via branca ———
legend_html = """
{% macro html(this, kwargs) %}
  <div style="
      position: fixed;
      bottom: 50px; left: 50px;
      background: white; padding: 8px;
      border: 2px solid grey;
      font-size: 12px;
      z-index: 9999;
  ">
    <div style="display:inline-block; font-size:18px; color:gold; margin-right:6px;">&#9733;</div> Top Sale<br>
    <i style="background: blue; width: 10px; height: 10px;
       display: inline-block; margin-right: 6px;"></i> All other sales
  </div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# ——— Display ———
m


In [39]:
m.save('index.html')

## Data snagger

In [40]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [41]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-06-03,Condo/Co-op,525 S Flagler Dr Ph 3ab,West Palm Beach,FL,33401.0,6050000.0,3.0,4.0,Plaza OF The Palm Beaches Condo,4018.0,NaN,1986.0,NaN,1506.0,5986.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/525-...,Beaches MLS,RX-11079097,N,Y,26.708264,-80.051289,https://www.redfin.com/FL/West-Palm-Beach/525-...,Samantha Curry,Douglas Elliman,None,None,Samantha Curry,Douglas Elliman,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [42]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_06092025


# CREATE TEMPLATE 

In [43]:
muni_set = set(df_top_ten['CITY'])

In [44]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [45]:
df_top_ten.reset_index(inplace=True,drop=True)

In [46]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [47]:
top_sale

'Plaza OF The Palm Beaches Condo at 525 S Flagler Dr Ph 3ab in West Palm Beach'

In [48]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [49]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [50]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-06-03,Condo/Co-op,525 S Flagler Dr Ph 3ab,West Palm Beach,FL,33401.0,6050000.0,3.0,4.0,Plaza OF The Palm Beaches Condo,4018.0,NaN,1986.0,NaN,1506.0,5986.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/525-...,Beaches MLS,RX-11079097,N,Y,26.708264,-80.051289,https://www.redfin.com/FL/West-Palm-Beach/525-...,Samantha Curry,Douglas Elliman,None,None,Samantha Curry,Douglas Elliman,None,None,1,orange,Plaza OF The Palm Beaches Condo at 525 S Flagl...
1,PAST SALE,2025-06-04,Condo/Co-op,20155 Boca West Dr Unit C601,Boca Raton,FL,33434.0,5300000.0,3.0,3.5,Akoya Boca West,2873.0,NaN,2019.0,NaN,1845.0,5133.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20155-Boc...,Beaches MLS,RX-11076823,N,Y,26.377468,-80.163463,https://www.redfin.com/FL/Boca-Raton/20155-Boc...,Rebecca Spooner,Siemens Group Realty Corp.,James Spooner,Siemens Group Realty Corp.,Judith Romanow,Boca West Realty LLC,None,None,2,blue,Akoya Boca West at 20155 Boca West Dr Unit C60...
2,PAST SALE,2025-06-04,Condo/Co-op,525 S Flagler Dr Unit 21 C/D,West Palm Beach,FL,33401.0,4900000.0,3.0,4.5,Plaza OF The Palm Beaches Condo,4018.0,NaN,1986.0,NaN,1220.0,5853.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/525-...,Beaches MLS,RX-11040645,N,Y,26.708413,-80.051135,https://www.redfin.com/FL/West-Palm-Beach/525-...,Christopher Leavitt,Douglas Elliman,None,None,Valentina Kraver,Douglas Elliman,None,None,3,blue,Plaza OF The Palm Beaches Condo at 525 S Flagl...
3,PAST SALE,2025-06-02,Condo/Co-op,2700 N Ocean Dr Unit 2502b,Riviera Beach,FL,33404.0,3100000.0,3.0,3.5,2700 North Ocean Condo,2660.0,NaN,2008.0,NaN,1165.0,3942.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-11053568,N,Y,26.785118,-80.033251,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Patrick Letourneau,One Sotheby's International Re,None,None,Michelle Rinaldi,EXP Realty LLC,None,None,4,blue,2700 North Ocean Condo at 2700 N Ocean Dr Unit...
4,PAST SALE,2025-06-03,Condo/Co-op,2727 Ocean Blvd #407,Highland Beach,FL,33487.0,2750000.0,3.0,2.5,Villa Magna Condo,2510.0,NaN,1982.0,NaN,1096.0,2823.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/2727-...,Beaches MLS,F10477372,N,Y,26.424944,-80.063184,https://www.redfin.com/FL/Highland-Beach/2727-...,Stephen Clementi,Coldwell Banker Realty,None,None,None,Sun Realty,None,None,5,blue,Villa Magna Condo at 2727 Ocean Blvd #407 in H...
5,PAST SALE,2025-06-05,Condo/Co-op,4475 N Ocean Blvd Unit 21d,Delray Beach,FL,33483.0,1532500.0,2.0,2.0,ST Andrews Ocean Condo,1520.0,NaN,1973.0,NaN,1008.0,2357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/4475-N-...,Beaches MLS,RX-11088899,N,Y,26.503852,-80.053038,https://www.redfin.com/FL/Delray-Beach/4475-N-...,Bettina Smith,The Corcoran Group,None,None,Bettina Smith,The Corcoran Group,None,None,6,blue,ST Andrews Ocean Condo at 4475 N Ocean Blvd Un...
6,PAST SALE,2025-06-04,Condo/Co-op,86 Macfarlane Dr Unit 5b,Delray Beach,FL,33483.0,1520000.0,3.0,3.0,Bar Harbour Condo,2363.0,NaN,1967.0,NaN,643.0,2887.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/86-Macf...,Beaches MLS,RX-11063198,N,Y,26.460259,-80.063221,https://www.redfin.com/FL/Delray-Beach/86-Macf...,Bettina Smith,The Corcoran Group,None,None,Bettina Smith,The Corcoran Group,None,None,7,blue,Bar Harbour Condo at 86 Macfarlane Dr Unit 5b ...
7,PAST SALE,2025-06-03,Condo/Co-op,3000 S Ocean Blvd #1501,Boca Raton,FL,33432.0,1250000.0,3.0,2.0,3000 South,1782.0,NaN,1970.0,NaN,701.0,2689.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/3000-S-Oc...,Be

In [51]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Plaza OF The Palm Beaches Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $985,000 to $6,050,000
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, Singer Island, Boca Raton, West Palm Beach, Riviera Beach, Highland Beach, Delray Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 91 condo sales totaling $53,926,699 million from May 30 to June 7. The previous week, brokers closed 129 condo sales totaling $62,644,186.

Last week’s units sold for an average of $592,601, higher t

In [52]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [53]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [54]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [55]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-06-03,Condo/Co-op,525 S Flagler Dr Ph 3ab,West Palm Beach,FL,33401.0,6050000.0,3.0,4.0,Plaza OF The Palm Beaches Condo,4018.0,NaN,1986.0,NaN,1506.0,5986.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/525-...,Beaches MLS,RX-11079097,N,Y,26.708264,-80.051289,https://www.redfin.com/FL/West-Palm-Beach/525-...,Samantha Curry,Douglas Elliman,None,None,Samantha Curry,Douglas Elliman,None,None,1,orange,Plaza OF The Palm Beaches Condo at 525 S Flagl...
1,PAST SALE,2025-06-04,Condo/Co-op,20155 Boca West Dr Unit C601,Boca Raton,FL,33434.0,5300000.0,3.0,3.5,Akoya Boca West,2873.0,NaN,2019.0,NaN,1845.0,5133.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20155-Boc...,Beaches MLS,RX-11076823,N,Y,26.377468,-80.163463,https://www.redfin.com/FL/Boca-Raton/20155-Boc...,Rebecca Spooner,Siemens Group Realty Corp.,James Spooner,Siemens Group Realty Corp.,Judith Romanow,Boca West Realty LLC,None,None,2,blue,Akoya Boca West at 20155 Boca West Dr Unit C60...
2,PAST SALE,2025-06-04,Condo/Co-op,525 S Flagler Dr Unit 21 C/D,West Palm Beach,FL,33401.0,4900000.0,3.0,4.5,Plaza OF The Palm Beaches Condo,4018.0,NaN,1986.0,NaN,1220.0,5853.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/525-...,Beaches MLS,RX-11040645,N,Y,26.708413,-80.051135,https://www.redfin.com/FL/West-Palm-Beach/525-...,Christopher Leavitt,Douglas Elliman,None,None,Valentina Kraver,Douglas Elliman,None,None,3,blue,Plaza OF The Palm Beaches Condo at 525 S Flagl...
3,PAST SALE,2025-06-02,Condo/Co-op,2700 N Ocean Dr Unit 2502b,Riviera Beach,FL,33404.0,3100000.0,3.0,3.5,2700 North Ocean Condo,2660.0,NaN,2008.0,NaN,1165.0,3942.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-11053568,N,Y,26.785118,-80.033251,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Patrick Letourneau,One Sotheby's International Re,None,None,Michelle Rinaldi,EXP Realty LLC,None,None,4,blue,2700 North Ocean Condo at 2700 N Ocean Dr Unit...
4,PAST SALE,2025-06-03,Condo/Co-op,2727 Ocean Blvd #407,Highland Beach,FL,33487.0,2750000.0,3.0,2.5,Villa Magna Condo,2510.0,NaN,1982.0,NaN,1096.0,2823.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/2727-...,Beaches MLS,F10477372,N,Y,26.424944,-80.063184,https://www.redfin.com/FL/Highland-Beach/2727-...,Stephen Clementi,Coldwell Banker Realty,None,None,None,Sun Realty,None,None,5,blue,Villa Magna Condo at 2727 Ocean Blvd #407 in H...
5,PAST SALE,2025-06-05,Condo/Co-op,4475 N Ocean Blvd Unit 21d,Delray Beach,FL,33483.0,1532500.0,2.0,2.0,ST Andrews Ocean Condo,1520.0,NaN,1973.0,NaN,1008.0,2357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/4475-N-...,Beaches MLS,RX-11088899,N,Y,26.503852,-80.053038,https://www.redfin.com/FL/Delray-Beach/4475-N-...,Bettina Smith,The Corcoran Group,None,None,Bettina Smith,The Corcoran Group,None,None,6,blue,ST Andrews Ocean Condo at 4475 N Ocean Blvd Un...
6,PAST SALE,2025-06-04,Condo/Co-op,86 Macfarlane Dr Unit 5b,Delray Beach,FL,33483.0,1520000.0,3.0,3.0,Bar Harbour Condo,2363.0,NaN,1967.0,NaN,643.0,2887.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/86-Macf...,Beaches MLS,RX-11063198,N,Y,26.460259,-80.063221,https://www.redfin.com/FL/Delray-Beach/86-Macf...,Bettina Smith,The Corcoran Group,None,None,Bettina Smith,The Corcoran Group,None,None,7,blue,Bar Harbour Condo at 86 Macfarlane Dr Unit 5b ...
7,PAST SALE,2025-06-03,Condo/Co-op,3000 S Ocean Blvd #1501,Boca Raton,FL,33432.0,1250000.0,3.0,2.0,3000 South,1782.0,NaN,1970.0,NaN,701.0,2689.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/3000-S-Oc...,Be

In [56]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Plaza OF The Palm Beaches Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $985,000 to $6,050,000
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, Singer Island, Boca Raton, West Palm Beach, Riviera Beach, Highland Beach, Delray Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 91 condo sales totaling $53,926,699 million from May 30 to June 7. The previous week, brokers closed 129 condo sales totaling $62,644,186.

Last week’s units sold for an average of $592,601, higher t

In [64]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/West-Palm-Beach/525-S-Flagler-Dr-33401/unit-3ab/home/173721812


In [65]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Boca-Raton/20155-Boca-West-Dr-33434/unit-C601/home/174312227


In [66]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Boca-Raton/2003-N-Ocean-Blvd-33431/unit-302/home/42316853


In [67]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Riviera-Beach/5150-N-Ocean-Dr-33404/unit-1803/home/42583216


In [68]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Boca-Raton/20155-Boca-West-Dr-33434/unit-C601/home/174312227


In [69]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Boca-Raton/2003-N-Ocean-Blvd-33431/unit-302/home/42316853


In [70]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-06-03,Condo/Co-op,525 S Flagler Dr Ph 3ab,West Palm Beach,FL,33401.0,6050000.0,3.0,4.0,Plaza OF The Palm Beaches Condo,4018.0,NaN,1986.0,NaN,1506.0,5986.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/525-...,Beaches MLS,RX-11079097,N,Y,26.708264,-80.051289,https://www.redfin.com/FL/West-Palm-Beach/525-...,Samantha Curry,Douglas Elliman,None,None,Samantha Curry,Douglas Elliman,None,None,1,orange,Plaza OF The Palm Beaches Condo at 525 S Flagl...
1,PAST SALE,2025-06-04,Condo/Co-op,20155 Boca West Dr Unit C601,Boca Raton,FL,33434.0,5300000.0,3.0,3.5,Akoya Boca West,2873.0,NaN,2019.0,NaN,1845.0,5133.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20155-Boc...,Beaches MLS,RX-11076823,N,Y,26.377468,-80.163463,https://www.redfin.com/FL/Boca-Raton/20155-Boc...,Rebecca Spooner,Siemens Group Realty Corp.,James Spooner,Siemens Group Realty Corp.,Judith Romanow,Boca West Realty LLC,None,None,2,blue,Akoya Boca West at 20155 Boca West Dr Unit C60...
2,PAST SALE,2025-06-04,Condo/Co-op,525 S Flagler Dr Unit 21 C/D,West Palm Beach,FL,33401.0,4900000.0,3.0,4.5,Plaza OF The Palm Beaches Condo,4018.0,NaN,1986.0,NaN,1220.0,5853.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/525-...,Beaches MLS,RX-11040645,N,Y,26.708413,-80.051135,https://www.redfin.com/FL/West-Palm-Beach/525-...,Christopher Leavitt,Douglas Elliman,None,None,Valentina Kraver,Douglas Elliman,None,None,3,blue,Plaza OF The Palm Beaches Condo at 525 S Flagl...
3,PAST SALE,2025-06-02,Condo/Co-op,2700 N Ocean Dr Unit 2502b,Riviera Beach,FL,33404.0,3100000.0,3.0,3.5,2700 North Ocean Condo,2660.0,NaN,2008.0,NaN,1165.0,3942.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-11053568,N,Y,26.785118,-80.033251,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Patrick Letourneau,One Sotheby's International Re,None,None,Michelle Rinaldi,EXP Realty LLC,None,None,4,blue,2700 North Ocean Condo at 2700 N Ocean Dr Unit...
4,PAST SALE,2025-06-03,Condo/Co-op,2727 Ocean Blvd #407,Highland Beach,FL,33487.0,2750000.0,3.0,2.5,Villa Magna Condo,2510.0,NaN,1982.0,NaN,1096.0,2823.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/2727-...,Beaches MLS,F10477372,N,Y,26.424944,-80.063184,https://www.redfin.com/FL/Highland-Beach/2727-...,Stephen Clementi,Coldwell Banker Realty,None,None,None,Sun Realty,None,None,5,blue,Villa Magna Condo at 2727 Ocean Blvd #407 in H...
5,PAST SALE,2025-06-05,Condo/Co-op,4475 N Ocean Blvd Unit 21d,Delray Beach,FL,33483.0,1532500.0,2.0,2.0,ST Andrews Ocean Condo,1520.0,NaN,1973.0,NaN,1008.0,2357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/4475-N-...,Beaches MLS,RX-11088899,N,Y,26.503852,-80.053038,https://www.redfin.com/FL/Delray-Beach/4475-N-...,Bettina Smith,The Corcoran Group,None,None,Bettina Smith,The Corcoran Group,None,None,6,blue,ST Andrews Ocean Condo at 4475 N Ocean Blvd Un...
6,PAST SALE,2025-06-04,Condo/Co-op,86 Macfarlane Dr Unit 5b,Delray Beach,FL,33483.0,1520000.0,3.0,3.0,Bar Harbour Condo,2363.0,NaN,1967.0,NaN,643.0,2887.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/86-Macf...,Beaches MLS,RX-11063198,N,Y,26.460259,-80.063221,https://www.redfin.com/FL/Delray-Beach/86-Macf...,Bettina Smith,The Corcoran Group,None,None,Bettina Smith,The Corcoran Group,None,None,7,blue,Bar Harbour Condo at 86 Macfarlane Dr Unit 5b ...
7,PAST SALE,2025-06-03,Condo/Co-op,3000 S Ocean Blvd #1501,Boca Raton,FL,33432.0,1250000.0,3.0,2.0,3000 South,1782.0,NaN,1970.0,NaN,701.0,2689.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/3000-S-Oc...,Be

## Time on Market Calculator

In [72]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2025, 3, 1) ## List (Earlier) date
date2 = datetime(2025, 6, 4) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

95


## Clean CSV for Datawrapper Chart

In [77]:
chart_df = df_top_ten

In [78]:
chart_df = chart_df.fillna(" ")

In [79]:
chart_df['LISTING AGENTS'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING AGENTS'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [80]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [82]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION",
    "BEDS", 
    "BATHS",
    "SOURCE", 
    "MLS#"
]

In [83]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [84]:
chart_df = chart_df.rename(columns={'$/SQUARE FEET':'PPSF'})

In [85]:
chart_df['PPSF'].astype(float)

0    1506.0
1    1845.0
2    1220.0
3    1165.0
4    1096.0
5    1008.0
6     643.0
7     701.0
8     565.0
9     483.0
Name: PPSF, dtype: float64

In [86]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['PPSF'] = chart_df['PPSF'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))

In [87]:
desired_order = [
    'SOLD DATE',
    'ADDRESS',
    'PRICE',
    'SQUARE FEET',
    'PPSF',
    'YEAR BUILT',
    'LISTING AGENTS',
    'BUYING AGENTS'
]

chart_df = chart_df[desired_order]


In [88]:
csv_date_string = today.strftime("%m_%d_%Y")

In [89]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [90]:
chart_df

,SOLD DATE,ADDRESS,PRICE,SQUARE FEET,PPSF,YEAR BUILT,LISTING AGENTS,BUYING AGENTS
0,2025-06-03,525 S Flagler Dr Ph 3ab in West Palm Beach,"$6,050,000","4,018","$1,506",1986,Samantha Curry Douglas Elliman,Samantha Curry Douglas Elliman
1,2025-06-04,20155 Boca West Dr Unit C601 in Boca Raton,"$5,300,000","2,873","$1,845",2019,Rebecca Spooner Siemens Group Realty Corp. Jam...,Judith Romanow Boca West Realty LLC
2,2025-06-04,525 S Flagler Dr Unit 21 C/D in West Palm Beach,"$4,900,000","4,018","$1,220",1986,Christopher Leavitt Douglas Elliman,Valentina Kraver Douglas Elliman
3,2025-06-02,2700 N Ocean Dr Unit 2502b in Riviera Beach,"$3,100,000","2,660","$1,165",2008,Patrick Letourneau One Sotheby's International...,Michelle Rinaldi EXP Realty LLC
4,2025-06-03,2727 Ocean Blvd #407 in Highland Beach,"$2,750,000","2,510","$1,096",1982,Stephen Clementi Coldwell Banker Realty,Sun Realty
5,2025-06-05,4475 N Ocean Blvd Unit 21d in Delray Beach,"$1,532,500","1,520","$1,008",1973,Bettina Smith The Corcoran Group,Bettina Smith The Corcoran Group
6,2025-06-04,86 Macfarlane Dr Unit 5b in Delray Beach,"$1,520,000","2,363",$643,1967,Bettina Smith The Corcoran Group,Bettina Smith The Corcoran Group
7,2025-06-03,3000 S Ocean Blvd #1501 in Boca Raton,"$1,250,000","1,782",$701,1970,Joseph Hillner Guaranteed Home Sales,"Joyce Marie Schneider Castles By The Beach, In..."
8,2025-06-03,5150 N Ocean Dr #1803 in Singer Island,"$1,180,000","2,088",$565,2003,Angela Voland Waterfront Properties & Club C,Jonathan Hazman Echo Fine Properties
9,2025-06-03,2003 N Ocean Blvd #302 in Boca Raton,"$985,000","2,038",$483,1980,Lisa Nanfra Compass Florida LLC,Deena Padnis Re/Max 1st Choice
